# Helpful links
- https://stackoverflow.com/questions/15007304/histogram-equalization-not-working-on-color-image-opencv
- https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3

In [ ]:
import os
import os.path as osp
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime

from sklearn.cluster import KMeans

scale_percent = 75  # percent of original size

In [ ]:
# DATA_PATH = r'/scratch/gallowaa/cciw/Data'  # MLRG
DATA_PATH = r'/media/angus/cciw/'  # Laptop

In [ ]:
# Search for all image files in testing set...
all_images = glob(
    osp.join(DATA_PATH, 'Data/Videos_and_stills/TestingSet/WHERD/*/*/*/*/*.jpg'))
print(len(all_images))

In [ ]:
#save_path = osp.join('/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test/WHERD')
save_path = osp.join(DATA_PATH, 'dataset_raw/Test/WHERD-unlabeled')
print(save_path)

In [ ]:
template = np.zeros([51, 51, 3]).astype('uint8')
#template = cv.circle(template, (25, 25), 25, (0, 0, 255), -1)
template = cv.ellipse(template, (25, 25), (10, 25), 0, 0, 360, (0, 0, 255), -1)
# plt.imshow(template)
w, h = template.shape[:2]

In [ ]:
"""Note: Names differ slightly from email with subject line 
'Selection of images to label for testing set' sent by 
dominique.brunet@canada.ca Wed 2/12/2020 4:04 PM, 
add 'FS' prefix to images taken with FishSense camera.
"""

# files to include
include_list = [
    'WHERD_1353_2019-10-30_GoPro-8.jpg',
    'WHERD_1355_2019-10-30_GoPro-5.jpg',
    'WHERD_FS1355_2019-11-03_FishSens-1.jpg',  # (100% mussel coverage)
    'WHERD_HP115410_2019-10-30_GoPro-3.jpg',
    'WHERD_HP131641_2019-11-03_GoPro-13.jpg',
    'WHERD_FSHP131642_2019-11-03_FishSens-4.jpg',
    'WHERD_FSHP131645_2019-11-03_FishSens-6.jpg',
    'WHERD_FSHP131649_2019-11-03_FishSens-2.jpg',
    'WHERD_LEE06021_2019-10-30_GoPro-3.jpg',
    'WHERD_FSLEE06022_2019-11-03_FishSens-1.jpg',
    'WHERD_LEE06031_2019-10-30_GoPro-2.jpg',
    'WHERD_LEE06032_2019-11-03_GoPro-3.jpg',
    'WHERD_LEE06033_2019-10-30_GoPro-8.jpg',
    'WHERD_FSLEE06037_2019-11-03_FishSens-1.jpg',
    'WHERD_LEE06070_2019-10-30_GoPro-1.jpg',  # (no mussels)
    'WHERD_LEE06075_2019-10-30_GoPro-11.jpg'
]
print(len(include_list))

In [ ]:
# meta-parameters for algorithm to find two red laser dots
# red_threshold is out of 255, match_threshold out of 1.0 for template matching
# red_threshold, match_threshold = 200, 0.2 # 10 correct
intensity_threshold, match_threshold = 150, 0.3  # 10

"""C - Constant subtracted from the mean or weighted mean (see the details below). 
Normally, it is positive but may be zero or negative as well."""
C = -15

"""Block size - Size of a pixel neighborhood that is used to calculate a threshold 
value for the pixel: 3, 5, 7, and so on."""
block_size = 201

# noise removing kernel
kernel = np.ones((5, 5), np.uint8)
dilate_kernel = np.ones((6, 6), np.uint8)

In [ ]:
j = 0
for i in range(len(all_images)):
    f = all_images[i]
    if f.split('/')[-1] in include_list:
        im = cv.imread(f)
        outfile = os.path.join(save_path, f.split('/')[-1])

        im = cv.imread(f)  # im will be in BGR format
        im_thresh = im.copy()
        
        im_ycrbc = cv.cvtColor(im, cv.COLOR_BGR2YCrCb)
        
        Y  = im_ycrbc[:, :, 0] # Separate pixel intensity Y from 
        Cr = im_ycrbc[:, :, 1] # red-difference chroma 
        Cb = im_ycrbc[:, :, 2] # blue-difference components
        
        im_thresh[Cr < intensity_threshold] = 0
        
        cts, bin_edges = np.histogram(im_ycrbc[:, :, 2])
        red_range = bin_edges.max() - bin_edges.min()
        
        Cr_th = cv.adaptiveThreshold(
            Cr, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 201, C)
        
        # morphology and connected components
        opening = cv.morphologyEx(Cr_th, cv.MORPH_OPEN, kernel, iterations=2)
        sure_bg = cv.dilate(opening, dilate_kernel, iterations=3)
        
        # Marker labelling
        ret, markers = cv.connectedComponents(sure_bg)
        
        # get the cluster IDs and number of pixels in each
        ids, cts = np.unique(markers, return_counts=True)
        
        # only keep the largest two clusters by pixel count
        largest_blobs = ids[1:][np.argsort(cts[1:])][-2:]
        
        if len(largest_blobs) == 2:
            mask = markers == largest_blobs[0]
            mask |= markers == largest_blobs[1]
            markers[np.invert(mask)] = 0
        
        # we can now encode clusters with the same value as
        # they should be physically separated
        markers[markers > 0] = 1

        # get numpy array of coordinates of non-zero elements
        coords = cv.findNonZero(markers)
        
        if coords is not None:
            kmeans = KMeans(n_clusters=2, random_state=0).fit(coords.squeeze())
            # draw the two clusters
            for pt in kmeans.cluster_centers_:
                _ = cv.circle(im, (int(pt[0]), int(pt[1])), 50, (0, 255, 0), 2)
            
            # compute the distance in pixels between the two laser beams
            d = np.linalg.norm(
                kmeans.cluster_centers_[0] - kmeans.cluster_centers_[1])
        else:
            d = 0

        fig, ax = plt.subplots(1, 1, figsize=(14, 6))
        title_str = '%d Ratio %.3f - dots are dist %d of %d px' % (j, d/im.shape[1], d, im.shape[1])
        ax.imshow(im)
        ax.set_title(title_str)
        plt.show()
        j += 1